# Simple power distribution optimisation example
This example includes the electical system, with the aim to optimise power supply and distribution in a cluster of platforms with varying power demand

In [13]:
%load_ext autoreload
%autoreload 2
from dataclasses import asdict
from oogeso.plots import plots
import oogeso.io.file_io
import IPython.display
import numpy as np
import pandas as pd
import logging
import pprint
import ipywidgets
import plotly.express as px
logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Read input data and create optimisation problem

In [14]:
#timerange=[0,400] # testing
time_range = [0,6]

data = oogeso.io.file_io.read_data_from_yaml('example_P1_powerdistribution.yaml')
#with open("skjold.json","w") as f:
#    f.write(serialize_oogeso_data(data))
    
# Creating simulation/optimisation object:
sim = oogeso.Simulator(data)

TypeError: __init__() got an unexpected keyword argument 'el_reserve_margin'

In [ ]:
@ipywidgets.interact(datagroup=['','devices','nodes','edges','carriers','parameters','profiles'])
def showdata(datagroup):
    pprint.pprint(asdict(data)[datagroup],indent=1) if datagroup!='' else print('')
yy=['']+list(sim.optimiser.component_objects(oogeso.optimiser.pyo.Constraint, active=True))
@ipywidgets.interact(constraint=yy)
def showdata(constraint):
    pprint.pprint(constraint.pprint(),width=1) if constraint!='' else print('')

In [ ]:
dotG=plots.plot_network(sim,timestep=None,filename=None,rankdir='LR')
IPython.display.Image(dotG.create_png())

In [ ]:
plots.plot_profiles(data.profiles).update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0))

In [ ]:
plotdata=pd.DataFrame()
for i,e in sim.optimiser.all_edges.items():
    ploss_func=e.edge_data.power_loss_function
    if ploss_func is not None:
        plotdata=pd.concat([plotdata,pd.DataFrame(data={i:ploss_func[1]},index=ploss_func[0])])
px.line(plotdata).update_layout(autosize=False,width=700,height=300,margin=dict(l=0,r=0,t=30,b=0),
    xaxis_title="Power transfer (MW)",yaxis_title="Power loss (MW)",legend_title="Edge").show()
plotdata={}
for d,dev in sim.optimiser.all_devices.items():
    if hasattr(dev.dev_data,"penalty_function"): 
        func=dev.dev_data.penalty_function
        plotdata[d]=(pd.Series(data=func[1],index=func[0]))
plotdata = pd.concat(plotdata,axis=1).interpolate("linear",limit_area="inside")
px.line(plotdata).update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0),
    xaxis_title="Power (MW)",yaxis_title="Penalty",legend_title="Device").show()

## Solve

In [ ]:
sim_result = sim.runSimulation(solver="cbc",timerange=time_range,write_yaml=False,timelimit=20)

In [ ]:
dotG=plots.plot_network(simulator=sim,timestep=1,filename=None,rankdir='LR')
IPython.display.Image(dotG.create_png())

## Analyse results

In [ ]:
plots.plot_sum_power_mix(sim_result,"el")

In [ ]:
gts = [d for d,d_obj in sim.optimiser.all_devices.items() if d_obj.dev_data.model=='powersource']
fig=plots.plot_device_profile(sim_result,devs=gts,include_on_off=True, include_prep=False)
fig.update_layout(autosize=False,width=800,height=400,margin=dict(l=0,r=0,t=30,b=0))

In [ ]:
dfplot = sim_result.dfPenalty#.unstack("device")
dfplot.index.name="Timestep"
dfplot.columns.name="Device"
px.area(dfplot,line_shape="hv",title="Penalty contribution per device"
    ).update_layout(legend_traceorder="reversed",yaxis_title="Penalty",
                    autosize=False,width=800,height=300,
                    margin=dict(l=0,r=0,t=30,b=0))

In [ ]:
plots.plot_reserve(sim_result).update_layout(autosize=False,width=800,height=300,margin=dict(l=0,r=0,t=30,b=0))

In [ ]:
px.line(sim_result.dfEdgeFlow.unstack("edge"),title="Edge flow",line_shape="hv").update_layout(
    autosize=False,width=700,height=300,margin=dict(l=0,r=0,t=30,b=0))

In [ ]:
#sim.optimiser.varEdgeLoss12["e1",10]

In [ ]:
#sim.optimiser.varDeviceIsOn.pprint()

In [ ]:
#sim.optimiser.varDevicePenalty["GT_A1","el","out",:].pprint()

In [ ]:
#sim.optimiser.varEdgeLoss.pprint()

In [ ]:
plots.plot_el_backup(sim_result, show_margin=True)

Thermal losses in conductors:

3-phase power: $P = \sqrt {3} V_L I_L \cos{\theta} \Rightarrow I_L = \frac{P}{V_L \sqrt{3}\cos{\theta}}$  
Total losses:  $P^{loss} = 3 I_L^2 R_L = \frac{P^2 R_L}{V_L^2 \cos{\theta}}$

In [ ]:
def cable_loss(power_flow_mw,voltage_kv,resistance_per_conductor_ohm,cos_theta=1):
    # TODO: check factors
    current_per_conductor_ka = power_flow_mw/(voltage_kv*np.sqrt(3)*cos_theta)
    losses=3*(current_per_conductor_ka**2 * resistance_per_conductor_ohm)
    return losses

In [ ]:
# losses 20 km cable, R=0.1 ohm/km per conductor
df=pd.DataFrame()
df["pwflow"]=range(0,51,5)
df["pwloss_10"] = df["pwflow"].apply(cable_loss,voltage_kv=10,resistance_per_conductor_ohm=0.1*20)
df["pwloss_20"] = df["pwflow"].apply(cable_loss,voltage_kv=20,resistance_per_conductor_ohm=0.1*20)
df["pwloss_30"] = df["pwflow"].apply(cable_loss,voltage_kv=30,resistance_per_conductor_ohm=0.1*20)
px.line(df.set_index("pwflow"))